In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import datetime

#Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

# sklearn stuff
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, Imputer 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

import feature_pipelines as pipes

### Submission Functions 

In [2]:
def generate_regression_preds(reg, model_name='pred_logerror', month=-1):
    reg_preds = None
    # change month of properties
    properties['month'] = month
    for i in range(int(properties.shape[0] / 100000)):   
        # get current test features
        current_test_feats = feature_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])

        # predict on current test obs
        current_preds = Series(reg.predict(current_test_feats), name=model_name,
                              index = np.arange(i*100000,(i+1)*100000))

        if reg_preds is not None:
            reg_preds = pd.concat([reg_preds, current_preds])
        else:
            reg_preds = current_preds

    #  fencepost problem
    current_test_feats = feature_pipeline.transform(properties.iloc[2900000:])
    current_preds = Series(reg.predict(current_test_feats), name=model_name,
                          index = np.arange(2900000,2985217))
    reg_preds = pd.concat([reg_preds, current_preds])
    del properties['month']
    return reg_preds

In [3]:
def generate_submissions(oct_model,nov_model,dec_model,name='new_submission',logy=True):
    """
    This function creates the submission file for the public leaderboard predictions.
    Three already fitted models, one for each of the predicting time points, is required.
    """
    submission_df = DataFrame()
    for i in range(int(properties.shape[0] / 100000)):
        all_feats = full_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])
        foo = properties.iloc[i*100000:(i+1)*100000][['parcelid']].reset_index(drop=True)
        if logy:
            foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                            '201611': nov_model.predict(all_feats),
                                                            '201612': dec_model.predict(all_feats)})], axis=1)
        else:
            foo = pd.concat([foo, DataFrame({'201610': np.log(oct_model.predict(all_feats)),
                                                            '201611': np.log(nov_model.predict(all_feats)),
                                                            '201612': np.log(dec_model.predict(all_feats))})], axis=1)
        submission_df = pd.concat([submission_df, foo], ignore_index=True)

    #  fencepost problem
    all_feats = full_pipeline.transform(properties.iloc[2900000:])
    foo = properties.iloc[2900000:][['parcelid']].reset_index(drop=True)
    foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                    '201611': nov_model.predict(all_feats),
                                                    '201612': dec_model.predict(all_feats)})], axis=1)
    submission_df = pd.concat([submission_df, foo], ignore_index=True)
    
    submission_df['201710'] = 0
    submission_df['201711'] = 0
    submission_df['201712'] = 0
    
    submission_df.rename(columns={'parcelid':'ParcelId'}, inplace=True)    
#     submission_df[['201610','201611','201612','201710','201711','201712']]= submission_df[['201610','201611','201612',
#                                                                                            '201710','201711','201712']].round(4)
    # unit test
    submission_df.drop_duplicates(inplace=True)
    assert submission_df.shape[0] == properties.shape[0]
    # write to .csv
    submission_df[['ParcelId','201610','201611','201612',
                  '201710','201711','201712']].to_csv(name + ".gz", index=False, float_format='%.4g', compression='gzip')
    return submission_df

In [4]:
def mean_absolute_errors(submission_df, comparison_df):
    """
    This function takes a submission entry for public leaderboard, and returns
    the training error for each month.
    """
    # training error
    trainresults = pd.merge(submission_df[['ParcelId','201610','201611','201612']], comparison_df[['parcelid','logerror','month']],
                           left_on='ParcelId', right_on='parcelid')
    oct_error = abs(trainresults[trainresults['month'] == 10]['201610'] 
                    - trainresults[trainresults['month'] == 10]['logerror']).mean()
    nov_error = abs(trainresults[trainresults['month'] == 11]['201611'] 
                    - trainresults[trainresults['month'] == 11]['logerror']).mean()
    dec_error = abs(trainresults[trainresults['month'] == 12]['201612'] 
                    - trainresults[trainresults['month'] == 12]['logerror']).mean()
    overall_mae = (oct_error*(trainresults['month'] == 10).sum() + nov_error*(trainresults['month'] == 11).sum() 
                        + dec_error*(trainresults['month'] == 12).sum()) / (trainresults['month'].isin([10,11,12])).sum()
    return (oct_error, nov_error, dec_error, overall_mae)

### Reading in data 

In [6]:
maindir = "/home/anerdi/Desktop/Zillow"

logerror = pd.read_csv(maindir + "/data/train_2016_v2.csv/train_2016_v2.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
properties = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv")

/home/anerdi/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#proportion of living area
properties['N-LivingAreaProp'] = properties['calculatedfinishedsquarefeet']/properties['lotsizesquarefeet']

properties['N-NonLivingAreaProp'] = properties['garagetotalsqft']/properties['lotsizesquarefeet']

#Ratio of the built structure value to land area
properties['N-ValueProp'] = properties['structuretaxvaluedollarcnt']/properties['landtaxvaluedollarcnt']

#Ratio of tax of property over parcel
properties['N-ValueRatio'] = properties['taxvaluedollarcnt']/properties['taxamount']

# Pool
properties['poolsizesum'] = properties['poolsizesum'].fillna(0)
# properties['Pool'] = (properties['poolsizesum'] > 0).astype(int)
properties['Pool'] = (properties['pooltypeid2'].fillna(0) + properties['pooltypeid7'].fillna(0)).astype(int)

properties['regionidcounty'] = properties['regionidcounty'].fillna(9999)
properties['regionidcity'] = properties['regionidcity'].fillna(-1)

# some more feature engineering
properties['age'] = 2017 - properties['yearbuilt']
properties['additional_rooms_count'] = np.maximum((properties['roomcnt'].values 
                                                   - properties['calculatedbathnbr'].values
                                                   - properties['bedroomcnt'].values),0)

### Data  Preprocessing Pipeline

In [8]:
# Setup variables considered in the model

# numerical variables
num_atts = ['bedroomcnt','calculatedbathnbr','age','additional_rooms_count',
           'calculatedfinishedsquarefeet','fullbathcnt','garagecarcnt','garagetotalsqft',
            'latitude','longitude','lotsizesquarefeet', 'roomcnt',
           'numberofstories','structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt','taxamount',
           'N-ValueRatio', 'N-LivingAreaProp', 'N-NonLivingAreaProp','N-ValueProp']

# categorical varaibles
cat_atts = ['airconditioningtypeid','architecturalstyletypeid',
           'buildingclasstypeid','heatingorsystemtypeid','Pool','propertylandusetypeid','regionidcounty',
           'storytypeid','typeconstructiontypeid','fireplaceflag','taxdelinquencyflag']

# Dictionary of categorical variables and their default levels
cat_dict = {'airconditioningtypeid':[-1] + list(range(1,14)),
           'architecturalstyletypeid':[-1] + list(range(1,28)),
           'buildingclasstypeid':[-1] + list(range(1,6)),
            'heatingorsystemtypeid':[-1] + list(range(1,26)),
            'pooltypeid10': list(range(-1,2)),
            'pooltypeid2': list(range(-1,2)),
            'pooltypeid7': list(range(-1,2)),
            'propertylandusetypeid': [-1, 31,46,47,246,247,248,260,261,262,263,264,265,266,267,268,269,270,271,
                                     273,274,275,276,279,290,291],
            'regionidcounty': [2061,3101,1286],
            'month': [-1] + list(range(1,13)),
            'storytypeid':[-1] + list(range(1,36)),
            'typeconstructiontypeid':[-1] + list(range(1,19)),
            'yearbuilt': [-1] + list(range(1885,2018)),
            'fireplaceflag': [-1] + ['True','False'],
            'taxdelinquencyflag': [-1] + ['Y','N']
           }

In [9]:
# Categorical pipeline
cat_pipeline = Pipeline([
        ('select_and_dummify', pipes.DF_Selector_GetDummies(cat_dict)),
    ])

# Numerical pipeline
num_pipeline = Pipeline([
        ('selector', pipes.DataFrameSelector(num_atts)),
        ('imputer', Imputer()),
    ])

# Full pipeline
feature_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline)
    ])

In [10]:
# impute missing num_atts per regionid
for countyid in properties.regionidcounty.unique():
    # setup condition
    cond = properties['regionidcounty'] == countyid
    indices = np.where(cond)[0]
    # impute values based on region
    if countyid != 9999:
        properties.loc[indices,num_atts] = (properties.loc[indices,num_atts]
                                .fillna(properties.loc[indices,num_atts]
                                .apply(np.mean)))
    else:
        properties.loc[indices,num_atts] = (properties.loc[indices,num_atts]
                                            .fillna(properties[num_atts]
                                            .apply(np.mean)))

In [11]:
assert properties[num_atts].isnull().any().any() == False

In [12]:
# join on parcel id
data = pd.merge(properties,logerror[['parcelid','logerror','month']], on='parcelid')
data['wts_rf_10'] = np.where(data['month'] == 10, 1.5, 1)
data['wts_rf_11'] = np.where(data['month'] == 11, 1.5, 1)
data['wts_rf_12'] = np.where(data['month'] == 12, 1.5, 1)

In [13]:
feature_pipeline.fit(data) #fitting the pipeline to the entire properties dataframe

FeatureUnion(n_jobs=1,
       transformer_list=[('num_pipeline', Pipeline(memory=None,
     steps=[('selector', DataFrameSelector(desired_cols=['bedroomcnt', 'calculatedbathnbr', 'age', 'additional_rooms_count', 'calculatedfinishedsquarefeet', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude', 'lotsizesqu...0, 48211.0, 3980.0, 37882.0, 47913.0, 36078.0, 19202.0], 'regionidcounty': [2061, 3101, 1286]}))]))],
       transformer_weights=None)

In [14]:
import gc

In [15]:
gc.collect()

69

## Splitting the Training Set

In [16]:
ix_overestimated = np.where(data['logerror'] >= 0)[0]
ix_underestimated = np.where(data['logerror'] < 0)[0]
data_indices = {"over": ix_overestimated, "under": ix_underestimated}

In [17]:
assert ix_overestimated.shape[0] + ix_underestimated.shape[0] == data.shape[0]

## Training Random Forests

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone

import warnings
warnings.filterwarnings("ignore")

In [19]:
models = [
    ("rf_maxdepth8",RandomForestRegressor(n_estimators = 100, max_features= 3, random_state=9, max_depth=8, criterion='mse')),
#     ("rf_maxdepth10",RandomForestRegressor(n_estimators = 100, max_features= 3, random_state=9, max_depth=10, criterion='mse')),
#     ("rf_maxdepth12",RandomForestRegressor(n_estimators = 100, max_features= 3, random_state=9, max_depth=12, criterion='mse')),
]

In [20]:
test_predictions = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv", usecols=['parcelid'])

for pair in models:
    current_model_name,current_model = pair
    print("Current model: %s" % current_model_name)
    
    for key,val in data_indices.items():
        type_of_zestimate, ix = key, val

        # preprocess current training data
        current_traindata = data.iloc[ix,]

        for month in [10,11,12]:
            print("%s_%d" % (type_of_zestimate, month))
            
            # get a clone of the model and fit the current training data
            reg = clone(current_model)
            reg.fit(feature_pipeline.transform(current_traindata), current_traindata['logerror'],
                   sample_weight = current_traindata['wts_rf_%d' % month])

            # obtain predictions on test set
            reg_preds = generate_regression_preds(reg, model_name="%s_%d_%s" % (current_model_name,month,type_of_zestimate),
                                                month = month)
            test_predictions = pd.concat([test_predictions, reg_preds], axis=1)

Current model: rf_maxdepth8
under_10
under_11
under_12
over_10
over_11
over_12


In [21]:
test_predictions.head()

,parcelid,rf_maxdepth8_10_under,rf_maxdepth8_11_under,rf_maxdepth8_12_under,rf_maxdepth8_10_over,rf_maxdepth8_11_over,rf_maxdepth8_12_over
0,10754147,-0.072228,-0.069010,-0.067635,0.077565,0.077129,0.077480
1,10759547,-0.072535,-0.069188,-0.067924,0.074611,0.074375,0.074987
2,10843547,-0.081216,-0.073977,-0.074956,0.112324,0.113053,0.112580
3,10859147,-0.074345,-0.073362,-0.073491,0.111927,0.112697,0.112555
4,10879947,-0.084413,-0.135045,-0.134260,0.112887,0.116730,0.116247


In [ ]:
test_predictions.to_csv("/home/anerdi/Desktop/Zillow/twostagemodel/RF-two-stage-preds.csv.gz",
                       compression='gzip', index=False)

In [22]:
overestimate_probabilities = pd.read_csv("/home/anerdi/Desktop/Zillow/twostagemodel/overestimate_probs_stacked_ann_rfs.csv.gz")
overestimate_probabilities.rename(columns={'stacked_pred':'overestimate_prob'}, inplace=True)

In [23]:
overestimate_probabilities.head()

,parcelid,ann_overestimate_prob,rf2_overestimate_prob,rf3_overestimate_prob,overestimate_prob
0,10754147,0.469632,0.466012,0.495383,0.479207
1,10759547,0.406392,0.555562,0.524675,0.475759
2,10843547,0.880731,0.548264,0.548752,0.732975
3,10859147,0.569588,0.663067,0.543329,0.602100
4,10879947,0.540791,0.519636,0.485341,0.531297


#### Merging Results

In [24]:
test_predictions = pd.merge(test_predictions, overestimate_probabilities, on='parcelid')

In [25]:
test_predictions.head()

,parcelid,rf_maxdepth8_10_under,rf_maxdepth8_11_under,rf_maxdepth8_12_under,rf_maxdepth8_10_over,rf_maxdepth8_11_over,rf_maxdepth8_12_over,ann_overestimate_prob,rf2_overestimate_prob,rf3_overestimate_prob,overestimate_prob
0,10754147,-0.072228,-0.069010,-0.067635,0.077565,0.077129,0.077480,0.469632,0.466012,0.495383,0.479207
1,10759547,-0.072535,-0.069188,-0.067924,0.074611,0.074375,0.074987,0.406392,0.555562,0.524675,0.475759
2,10843547,-0.081216,-0.073977,-0.074956,0.112324,0.113053,0.112580,0.880731,0.548264,0.548752,0.732975
3,10859147,-0.074345,-0.073362,-0.073491,0.111927,0.112697,0.112555,0.569588,0.663067,0.543329,0.602100
4,10879947,-0.084413,-0.135045,-0.134260,0.112887,0.116730,0.116247,0.540791,0.519636,0.485341,0.531297


In [26]:
for pair in models:
    current_model_name, current_model = pair
    # combine over and under to get prediction
    for month in [10,11,12]:
        test_predictions['{0}_{1}'.format(current_model_name, month)] = (test_predictions['%s_%d_over' % (current_model_name, month)]*test_predictions['overestimate_prob'] 
                + test_predictions['%s_%d_under' % (current_model_name, month)]*(1 - test_predictions['overestimate_prob']))

In [27]:
test_predictions.head()

,parcelid,rf_maxdepth8_10_under,rf_maxdepth8_11_under,rf_maxdepth8_12_under,rf_maxdepth8_10_over,rf_maxdepth8_11_over,rf_maxdepth8_12_over,ann_overestimate_prob,rf2_overestimate_prob,rf3_overestimate_prob,overestimate_prob,rf_maxdepth8_10,rf_maxdepth8_11,rf_maxdepth8_12
0,10754147,-0.072228,-0.069010,-0.067635,0.077565,0.077129,0.077480,0.469632,0.466012,0.495383,0.479207,-0.000446,0.001021,0.001905
1,10759547,-0.072535,-0.069188,-0.067924,0.074611,0.074375,0.074987,0.406392,0.555562,0.524675,0.475759,-0.002529,-0.000886,0.000068
2,10843547,-0.081216,-0.073977,-0.074956,0.112324,0.113053,0.112580,0.880731,0.548264,0.548752,0.732975,0.060644,0.063111,0.062503
3,10859147,-0.074345,-0.073362,-0.073491,0.111927,0.112697,0.112555,0.569588,0.663067,0.543329,0.602100,0.037809,0.038664,0.038527
4,10879947,-0.084413,-0.135045,-0.134260,0.112887,0.116730,0.116247,0.540791,0.519636,0.485341,0.531297,0.020412,-0.001278,-0.001166


In [28]:
model_name = 'rf_maxdepth8'
new_submission = DataFrame({'ParcelId': test_predictions['parcelid'],
                           '201610':test_predictions['%s_10' % model_name],
                           '201611':test_predictions['%s_11' % model_name],
                           '201612':test_predictions['%s_12' % model_name],
})
new_submission['201710'] = 0
new_submission['201711'] = 0
new_submission['201712'] = 0

In [29]:
# two-stage with rf
mean_absolute_errors(new_submission.round(4), data)

(0.06208370504319887,
 0.06111599123767798,
 0.07373208740655553,
 0.064248243970967081)

In [32]:
# two-stage with rf
mean_absolute_errors(new_submission.round(4), data)

(0.062054812135824734,
 0.061131763417305744,
 0.07362794709603225,
 0.064213579957855316)

In [29]:
new_submission.round(4).to_csv("/home/anerdi/Desktop/Zillow/submissions/two_stage_stage1_annrfs_stage2_rf_age.csv.gz", index=False,
                     compression='gzip')